In [9]:
from InitialGraphs import Graphdata
from InitialPattern2 import Pattern
from GraphPattern import PatternMarge
import time
import os

def Top_nPercent(dict, n):
    if len(dict) == 0:
        return 0
    part_v = len(dict)*n//100

    value = list(sorted(dict.items(), key = lambda item: item[1], reverse = True))[part_v][1]
    return value

fileName = 'NCI1'
pers = 10 # N-hop Pattern Filtering Ratio
psize = 100 # 𝐹𝑟𝑒𝑞𝑢𝑒𝑛𝑡 𝑃𝑎𝑡𝑡𝑒𝑟𝑛 𝑀𝑎𝑛𝑎𝑔𝑚𝑒𝑛𝑡 Table Size
batchsize = 3
thlist1 = []
DxMtrix = {}
ExceptionList = {}
iterations = 0
iterations2 = 0
FPMT = {}
th2 = batchsize * batchsize

for i in range(11):
    s1 = time.time()
    IGPath = './Gen_Evolution_set/' + fileName + '/' + fileName + '_' + str(i) + '.edges'
    lbPath = './dataset/' + fileName + '/' + fileName + '.node_labels'

    Separator = ','

    DataT0 = Graphdata(IGPath, lbPath, Separator)
    Th1 = Top_nPercent(DataT0.OhP, pers)
    
    iterations = i%batchsize
    
    if iterations < batchsize:
        FP = DataT0.OhP

        for ohp in FP.keys():
            ohpRe = ohp.split('/')
            ohpRe.reverse()
            ohpReverse = '/'.join(ohpRe)
            
            if ohp not in DxMtrix or ohpReverse not in DxMtrix:
                Raw = [[0 for col in range(batchsize)] for row in range(batchsize)]
                if FP[ohp] >= Th1:
                    Raw[iterations2][iterations] = 1
                    DxMtrix[ohp] = Raw
            else:
                if FP[ohp] >= Th1:
                    if ohpReverse in DxMtrix:
                        DxMtrix[ohpReverse][iterations2][iterations] = 1
                    else:
                        DxMtrix[ohp][iterations2][iterations] = 1

        if iterations == batchsize-1:
            for ohp in DxMtrix:
                if ohp not in FPMT:
                    Raw2 = [0 for col in range(batchsize+2)]
                    Raw2[iterations2] = sum(DxMtrix[ohp][iterations2])
                    FPMT[ohp] = Raw2
                else:
                    if FPMT[ohp][batchsize+1] != 0:
                        FPMT[ohp][iterations2] = sum(DxMtrix[ohp][iterations2])
            
            for nhp in FPMT:
                if FPMT[nhp][batchsize+1] != 0:
                    FPMT[nhp][iterations2] = 0
                    FPMT[nhp][batchsize+1] = 0
                    FPMT[nhp][batchsize] = sum(FPMT[nhp][:batchsize])
                    ExceptionList[nhp] = FPMT[nhp]
                else:
                    FPMT[nhp][batchsize] = sum(FPMT[nhp][:batchsize])
                    if FPMT[nhp][batchsize] <= int(th2*0.3):
                        FPMT[nhp][batchsize+1] = -1
                    elif FPMT[nhp][batchsize] >= int(th2*0.7):
                        FPMT[nhp][batchsize+1] = 1
            
            if i >= batchsize-1:
                iterations2 = 0
                DataP0 = Pattern(FPMT, DxMtrix, ExceptionList, psize)
                FPMT = DataP0.NhP
            
            iterations2 = iterations2 + 1
            
            
    
    PM = PatternMarge(DataT0.IG, FPMT, DataT0.lbDict, DataT0.nList)
    Ge = PM.SG

    e1 = time.time() - s1
    f = open('./Result/' + fileName + '_' + str(len(DataP0.NhP)) + '_' + str(pers) + '_' + str(i) + '.edges', 'w')

    for e in Ge:
        str_e = e[0] + ',' + e[1] + '\n'
        f.write(str_e)
            
    f.close()

    P_id_list = list(sorted(PM.pidDict.items(), key = lambda item: item[1], reverse = True))
    Gp = P_id_list
    f = open('./Result/' + fileName + '_' + str(len(DataP0.NhP)) + '_' + str(pers) + '_' + str(i) + '.pattern', 'w')

    for e in Gp:
        str_e = e[1] + ',' + e[0] + '\n'
        f.write(str_e)
            
    f.close()

    Gl = list(sorted(PM.SGnl.items()))
    f = open('./Result/' + fileName + '_' + str(len(DataP0.NhP)) + '_' + str(pers) + '_' + str(i) + '.node_labels', 'w')

    for e in Gl:
        str_e = e[0] + ',' + e[1] + '\n'
        f.write(str_e)
            
    f.close()

    print(Th1)
    print(len(FPMT))
    print(len(PM.SGnl))
    print(len(PM.SG))

    def convert_size(size_bytes):
        import math
        if size_bytes == 0:
            return "0B"
        size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
        i = int(math.floor(math.log(size_bytes, 1024)))
        p = math.pow(1024, i)
        s = round(size_bytes / p, 2)
        return "%s %s" % (s, size_name[i])

    file_name = './Result/' + fileName + '_' + str(len(DataP0.NhP)) + '_' + str(pers) + '_' + str(i)
    extension = ['.edges', '.node_labels', '.pattern']
    file_size = 0
    for e in extension:
        file_size = file_size + os.path.getsize(file_name + e) 
        print(e.split('.')[1], convert_size(os.path.getsize(file_name + e)))
        
    print('Sum File Size:', convert_size(file_size), 'bytes')

    input_file = [IGPath, lbPath]
    file_size1 = 0
    for ip in input_file:
        file_size1 = file_size1 + os.path.getsize(ip)

1 {'3/1': [2, 0, 0, 2], '3/3': [3, 0, 0, 3], '3/2': [1, 0, 0, 1], '5/3': [1, 0, 0, 1], '3/4': [1, 0, 0, 1], '7/3': [1, 0, 0, 1], '2/1': [1, 0, 0, 1], '2/3': [2, 0, 0, 2], '1/3': [1, 0, 0, 1], '1/2': [2, 0, 0, 2], '4/3': [1, 0, 0, 1]}
2 {'3/1': [2, 1, 0, 3], '3/3': [3, 3, 0, 6], '3/2': [1, 3, 0, 4], '5/3': [1, 1, 0, 2], '3/4': [1, 0, 0, 1], '7/3': [1, 1, 0, 2], '2/1': [1, 1, 0, 2], '2/3': [2, 0, 0, 2], '1/3': [1, 2, 0, 3], '1/2': [2, 2, 0, 4], '4/3': [1, 3, 0, 4], '2/2': [0, 1, 0, 1], '3/7': [0, 1, 0, 1], '3/5': [0, 1, 0, 1]}
3 {'3/1': [2, 1, 1, 4], '3/3': [3, 3, 3, 9], '3/2': [1, 3, 2, 6], '5/3': [1, 1, 2, 4], '3/4': [1, 0, 0, 1], '7/3': [1, 1, 2, 4], '2/1': [1, 1, 2, 4], '2/3': [2, 0, 1, 3], '1/3': [1, 2, 2, 5], '1/2': [2, 2, 1, 5], '4/3': [1, 3, 3, 7], '2/2': [0, 1, 0, 1], '3/7': [0, 1, 1, 2], '3/5': [0, 1, 1, 2]}


TypeError: can only concatenate str (not "int") to str

In [ ]:
th2 = 50
print(int(th2*0.1))
print(int(th2*0.2))
print(int(th2*0.3))
print()
print(int(th2*0.4))
print(int(th2*0.5))
print(int(th2*0.6))
print()
print(int(th2*0.7))
print(int(th2*0.8))
print(int(th2*0.9))
print(th2*1)